## The Battle of Neighborhoods
### Location Recommendation For Opening Italian Restaurant in Toronto


This is a capstone project for IBM Data Science Professional Certificate. In this project, I am creating a hypothetical scenario for a concept that there may not be enough Italian Restaurants in Toronto Area. Therefore it might be a great opportunity for an entrepreneur who is based in Canada.
As Italian food is popular among Canidian community, so this entrepreneur might think of opening its business in areas where the Canidian community resides. With the purpose in mind, finding the location to open such a restaurant is one of the most important decisions for this entrepreneur and I am designing this project to help him find the most suitable location.

### BUSINESS PROBLEM
The objective of this capstone project is to find the most suitable location for the entrepreneur to open a new Italian Restaurant in Toronto, Canada. By using data science methods and tools along with machine learning algorithms such as clustering, this project aims to provide solutions to answer the business question: In Toronto, if an entrepreneur wants to open an Italian Restaurant, where should they consider opening it?

### TARGET AUDIENCE
The entrepreneur who wants to find the location to open an authentic Italian restaurant.

### DATA
To solve this problem, we will need below data:   
List of neighborhoods in Toronto, Canada   
Latitude and Longitude of these neighborhoods   
Venue data related to Italian restaurants. This will help us find neighborhoods that are more suitable to open an Italian Restaurant.

### EXTRACTING THE DATA
The scrapping of Toronto neighborhoods via Wikipedia
Getting Latitude and Longitude data of these neighborhoods via Geocoder package
Using Foursquare API to get venue data related to these neighborhoods

### METHODOLOGY
First, I need to get the list of neighborhoods in Toronto, Canada. This is possible by extracting the list of neighborhoods from Wikipedia: ​https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M​   
I did the web scraping by utilizing pandas HTML table scraping method as it is easier and more convenient to pull tabular data directly from a web page into the data frame.   
However, it is only a list of neighborhood names and postal codes. I need to get their coordinates to utilize Foursquare to pull the list of venues near these neighborhoods. To get the coordinates, I tried using Geocoder Package but it was not working so I used the CSV file provided by IBM team to match the coordinates of Toronto neighborhoods. After gathering these coordinates, I visualize the map of Toronto using Folium package to verify whether these are correct coordinates. Next, I use Foursquare API to pull the list of top 100 venues within 500 meters radius. I have created a Foursquare developer account in order to obtain account ID and API key to pull the data. From Foursquare, I am able to pull the names, categories, latitude, and longitude of the venues. With this data, I can also check how many unique categories that I can get from these venues. Then, I analyze each neighborhood by grouping the rows by neighborhood and taking the mean on the frequency of occurrence of each venue category. This is to prepare clustering to be done later.
Here, I made a justification to specifically look for “Italian restaurants”. Lastly, I performed the clustering method by using k-means clustering. K-means clustering algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster while keeping the centroids as small as possible. It is one of the simplest and popular unsupervised machine learning algorithms and it is highly suited for this project as well. I have clustered the neighborhoods in Toronto into 3 clusters based on their frequency of occurrence for “Iranian food”. Based on the results (the concentration of clusters), I will be able to recommend the ideal location to open the restaurant.

#### RESULT
The results from k-means clustering show that we can categorize Toronto neighborhoods into 3 clusters based on how many Italian restaurants are in each neighborhood:   
 Cluster 0: Neighborhoods with a high number of Italian restaurants.   
 Cluster 1: Neighborhoods with a little number of Italian restaurants.   
 Cluster 2: Neighborhoods with a rare Italian restaurant.   


Let's start the project by importig libraries.

In [153]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py37_0
  - defaults/win-64::conda-4.8.1-py37_0done

# All requested packages already installed.

Libraries imported.



 We Use the BeautifulSoup package or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe

In [154]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [155]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xg1lggpAAEMAAHzLN7wAAADJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":933624196,"wgRevisionId":933624196,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

#### As we can see  the tabular data is availabe in table and belongs to class="wikitable sortable"So let's extract only table

In [156]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_

In [157]:
print(My_table.tr.text)


Postcode
Borough
Neighborhood



In [158]:
headers="Postcode,Borough,Neighbourhood"

#### Geting all values in tr and seperating each td within by ","

In [159]:
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

#### writing  data into csv file


In [162]:
file=open("toronto.csv","wb")

file.write(bytes(table1,encoding="ascii",errors="ignore"))

8708

#### Converting into dataframe

In [163]:
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned"¶ 

In [164]:
indexNames = df[ df['Borough'] =='Not assigned'].index
df.drop(indexNames , inplace=True)
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough¶ 

In [165]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### rows with the same postalcode will combined into one row with the neighborhoods separated with a comma

In [166]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head(20)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [167]:
df_new.shape

(103, 3)


### Use the Geocoder package or the csv file to create dataframe with longitude and latitude values

#### We will be using a csv file : http://cocl.us/Geospatial_data

In [168]:

address="C:/Users/HP/Desktop/Geospatial_Coordinates.csv"

In [169]:
df_lon_lat = pd.read_csv(address)
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [170]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [171]:
df_lon_lat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [129]:
Toronto_df = pd.merge(df_new,
                 df_lon_lat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
Toronto_df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [172]:
Toronto_df.shape

(103, 5)


### Explore and cluster the neighborhoods in Toronto

#### Use geopy library to get the latitude and longitude values of Toronto.


In [53]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [173]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [174]:
CLIENT_ID = 'GUYZYKE5JZINZVRK1A10BAFI33FXPBTTAUCG22I2CHSFYEQO' # your Foursquare ID
CLIENT_SECRET = '04RI121NVRR44VH5DYEU23QNC4RGHTM5H1A2LN0LDYPG5Z11' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GUYZYKE5JZINZVRK1A10BAFI33FXPBTTAUCG22I2CHSFYEQO
CLIENT_SECRET:04RI121NVRR44VH5DYEU23QNC4RGHTM5H1A2LN0LDYPG5Z11


In [175]:
radius=500
LIMIT=100

In [176]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighbourhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [177]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [178]:
toronto_venues.shape

(2219, 7)

#### Let's check how many venues were returned for each neighborhood

In [62]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,57,57,57,57,57,57


In [179]:
toronto_venues['Venue Category'].value_counts()[:50]

Coffee Shop                      194
Café                             102
Restaurant                        60
Pizza Place                       51
Park                              49
Bakery                            48
Italian Restaurant                46
Sandwich Place                    41
Hotel                             40
Bar                               40
Japanese Restaurant               37
Gym                               34
Clothing Store                    34
Burger Joint                      30
Fast Food Restaurant              28
Sushi Restaurant                  27
Grocery Store                     25
Breakfast Spot                    25
American Restaurant               25
Gastropub                         24
Pub                               23
Steakhouse                        23
Seafood Restaurant                23
Vegetarian / Vegan Restaurant     21
Chinese Restaurant                21
Pharmacy                          20
Beer Bar                          20
T

## Analysing Each Neighborhood

In [180]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [181]:
toronto_onehot.shape

(2219, 272)

In [109]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.010000,0.00000

Checking number of places having Mediterranean Restaurant as VenueCategory

In [182]:
len(toronto_grouped[toronto_grouped["Italian Restaurant"] > 0])

25

Creating dataframe with Mediterranean Restaurant data only


In [183]:
toronto_Italian_Restaurant = toronto_grouped[["Neighbourhood","Italian Restaurant"]]
toronto_Italian_Restaurant.head(10)

,Neighbourhood,Italian Restaurant
0,"Adelaide, King, Richmond",0.000000
1,Agincourt,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000
4,"Alderwood, Long Branch",0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000
6,Bayview Village,0.000000
7,"Bedford Park, Lawrence Manor East",0.086957
8,Berczy Park,0.000000
9,"Birch Cliff, Cliffside West",0.000000


In [186]:
# set number of clusters
kclusters = 3
toronto_clustering = toronto_Italian_Restaurant.drop(["Neighbourhood"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0])

In [187]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_Italian_Restaurant.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

In [188]:
toronto_merged.head(20)

,Neighbourhood,Italian Restaurant,Cluster Labels
0,"Adelaide, King, Richmond",0.000000,0
1,Agincourt,0.000000,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0
4,"Alderwood, Long Branch",0.000000,0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0
6,Bayview Village,0.000000,0
7,"Bedford Park, Lawrence Manor East",0.086957,2
8,Berczy Park,0.000000,0
9,"Birch Cliff, Cliffside West",0.000000,0


Joinging dataframes to get the latitude and longitude for corresponding neighborhood


In [189]:
toronto_merged = toronto_merged.join(Toronto_df.set_index("Neighbourhood"), on="Neighbourhood")

print(toronto_merged.shape)
toronto_merged.head(20)

(100, 7)


,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",0.000000,0,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,0.000000,0,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood, Long Branch",0.000000,0,M8W,Etobicoke,43.602414,-79.543484
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0,M3H,North York,43.754328,-79.442259
6,Bayview Village,0.000000,0,M2K,North York,43.786947,-79.385975
7,"Bedford Park, Lawrence Manor East",0.086957,2,M5M,North York,43.733283,-79.419750
8,Berczy Park,0.000000,0,M5E,Downtown Toronto,43.644771,-79.373306
9,"Birch Cliff, Cliffside West",0.000000,0,M1N,Scarborough,43.692657,-79.264848


Sorting dataframe based on cluster labels


In [190]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(100, 7)


,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",0.000000,0,M5H,Downtown Toronto,43.650571,-79.384568
67,"Moore Park, Summerhill East",0.000000,0,M4T,Central Toronto,43.689574,-79.383160
66,"Maryvale, Wexford",0.000000,0,M1R,Scarborough,43.750072,-79.295849
65,"Little Portugal, Trinity",0.017857,0,M6J,West Toronto,43.647927,-79.419750
64,Leaside,0.000000,0,M4G,East York,43.709060,-79.363452
63,Lawrence Park,0.000000,0,M4N,Central Toronto,43.728020,-79.388790
62,"Lawrence Heights, Lawrence Manor",0.000000,0,M6A,North York,43.718518,-79.464763
61,L'Amoreaux West,0.000000,0,M1W,Scarborough,43.799525,-79.318389
60,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0,M8Z,Etobicoke,43.628841,-79.520999
68,North Toronto West,0.000000,0,M4R,Central Toronto,43.715383,-79.405678


In [191]:
toronto_merged=toronto_merged.dropna()

In [192]:
toronto_merged

,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",0.000000,0,M5H,Downtown Toronto,43.650571,-79.384568
67,"Moore Park, Summerhill East",0.000000,0,M4T,Central Toronto,43.689574,-79.383160
66,"Maryvale, Wexford",0.000000,0,M1R,Scarborough,43.750072,-79.295849
65,"Little Portugal, Trinity",0.017857,0,M6J,West Toronto,43.647927,-79.419750
64,Leaside,0.000000,0,M4G,East York,43.709060,-79.363452
63,Lawrence Park,0.000000,0,M4N,Central Toronto,43.728020,-79.388790
62,"Lawrence Heights, Lawrence Manor",0.000000,0,M6A,North York,43.718518,-79.464763
61,L'Amoreaux West,0.000000,0,M1W,Scarborough,43.799525,-79.318389
60,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0,M8Z,Etobicoke,43.628841,-79.520999
68,North Toronto West,0.000000,0,M4R,Central Toronto,43.715383,-79.405678


#### Finally, let's visualize the resulting clusters

In [193]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [201]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters
### Cluster 0

In [202]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",0.000000,0,M5H,Downtown Toronto,43.650571,-79.384568
67,"Moore Park, Summerhill East",0.000000,0,M4T,Central Toronto,43.689574,-79.383160
66,"Maryvale, Wexford",0.000000,0,M1R,Scarborough,43.750072,-79.295849
65,"Little Portugal, Trinity",0.017857,0,M6J,West Toronto,43.647927,-79.419750
64,Leaside,0.000000,0,M4G,East York,43.709060,-79.363452
63,Lawrence Park,0.000000,0,M4N,Central Toronto,43.728020,-79.388790
62,"Lawrence Heights, Lawrence Manor",0.000000,0,M6A,North York,43.718518,-79.464763
61,L'Amoreaux West,0.000000,0,M1W,Scarborough,43.799525,-79.318389
60,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0,M8Z,Etobicoke,43.628841,-79.520999
68,North Toronto West,0.000000,0,M4R,Central Toronto,43.715383,-79.405678


In [205]:
len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 0])

81

### Cluster 1

In [203]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
85,"The Beaches West, India Bazaar",0.045455,1,M4L,East Toronto,43.668999,-79.315572
11,"Brockton, Exhibition Place, Parkdale Village",0.041667,1,M6K,West Toronto,43.636847,-79.428191
45,"Flemingdon Park, Don Mills South",0.043478,1,M3C,North York,43.725900,-79.340923
15,"Cabbagetown, St. James Town",0.046512,1,M4X,Downtown Toronto,43.667967,-79.367675
73,Queen's Park,0.024390,1,M9A,Queen's Park,43.667856,-79.532242
73,Queen's Park,0.024390,1,M7A,Downtown Toronto,43.662301,-79.389494
27,"Commerce Court, Victoria Hotel",0.030000,1,M5L,Downtown Toronto,43.648198,-79.379817
52,"High Park, The Junction South",0.045455,1,M6P,West Toronto,43.661608,-79.464763
51,"Harbourfront East, Toronto Islands, Union Station",0.040000,1,M5J,Downtown Toronto,43.640816,-79.381752
82,Studio District,0.046512,1,M4M,East Toronto,43.659526,-79.340923


In [206]:
len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 1])

11

### Cluster 2

In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighbourhood,Italian Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
77,"Runnymede, Swansea",0.058824,2,M6S,West Toronto,43.651571,-79.484450
19,Central Bay Street,0.059524,2,M5G,Downtown Toronto,43.657952,-79.387383
86,"The Danforth West, Riverdale",0.069767,2,M4K,East Toronto,43.679557,-79.352188
71,"Parkdale, Roncesvalles",0.076923,2,M6R,West Toronto,43.648960,-79.456325
7,"Bedford Park, Lawrence Manor East",0.086957,2,M5M,North York,43.733283,-79.419750
21,Christie,0.058824,2,M6G,Downtown Toronto,43.669542,-79.422564
24,"Clarks Corners, Sullivan, Tam O'Shanter",0.090909,2,M1T,Scarborough,43.781638,-79.304302
28,Davisville,0.062500,2,M4S,Central Toronto,43.704324,-79.388790


In [208]:
len(toronto_merged.loc[toronto_merged['Cluster Labels'] == 2])

8

## Observation

Most of the Italian restaurants are in cluster 0 which is around The Adelaide, King, Richmond, Moore Park, Summerhill East, Maryvale, Wexford Little Portugal, Trinity, Leaside, Lawrence Park and lowest in Cluster 2 areas which are in Runnymede, Swansea, Central Bay Street, The Danforth West, Riverdale. Also, there are good opportunities to open Davisville, Clarks Corners, Sullivan, Tam O'Shanter, Bedford Park, Lawrence Manor East. Looking at nearby venues it seems cluster 1might be a good location as there are not a lot of Italian restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Italian restaurant in these locations.